In [1]:
import os
import glob
import numpy as np
import cv2
from PIL import Image
import torch

from ultralytics import YOLO

import config.config as cfg
import params
import utils

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

print("Device: ", params.DEVICE)

Device:  cuda


In [2]:
# Initialize YOLO model
model = YOLO(cfg.YOLO_SEGMENTATION_MODEL)

In [6]:
images_with_errors = []

# Process each island
for island_path in glob.glob(f"{cfg.RAW_IMAGES_PATH}/*"):
    island_name = os.path.basename(island_path)
    print(f"Processing island {island_name}")

    # Process each lizard in the island
    for lizard_path in glob.glob(f"{island_path}/*"):
        torch.cuda.empty_cache()
        lizard_name = os.path.basename(lizard_path)
        # print(f"Processing lizard {lizard_name}")

        # Create a directory to store the processed images
        os.makedirs(
            f"{cfg.PROCESSED_IMAGES_PATH}/{island_name}/{lizard_name}", exist_ok=True
        )

        # Get all image paths for the lizard
        lizard_images = glob.glob(f"{lizard_path}/*.jpg")

        if len(lizard_images) == 0:
            print(f"No images found for lizard {lizard_name}")
            continue

        # if len(lizard_images) == 1:
        #     continue

        # Prepare two lists to store resized and processed images
        resized_image_paths = []
        processed_image_paths = []

        for image_path in lizard_images:
            # Get the basename of the image
            image_basename = os.path.basename(image_path)

            # Add the path of the resized image to the list
            processed_image_name = f"{os.path.splitext(image_basename)[0]}.png"
            processed_image_path = f"{cfg.PROCESSED_IMAGES_PATH}/{island_name}/{lizard_name}/{processed_image_name}"

            if os.path.exists(processed_image_path):
                continue

            print(f"Processing image {image_path}")
            # Read the image
            image = cv2.imread(image_path)

            # Resize the image
            resized_image = cv2.resize(image, (640, 640))

            # Add the paths to the lists
            resized_image_paths.append(image_basename)
            processed_image_paths.append(processed_image_path)

            # Save the resized image temporarily
            cv2.imwrite(image_basename, resized_image)

        if len(resized_image_paths) == 0:
            continue

        # Run YOLO model on the resized images
        results = model(resized_image_paths)

        # Extract masks and crop them from the results

        for idx, result in enumerate(results):
            try:
                boxes = result.boxes.xyxy.cpu().numpy()  # Get bounding boxes
                mask = result.masks.data.cpu().numpy()[0]  # Get the single mask
                orig_img = result.orig_img  # Get original image

                # Get the coordinates of the bounding box
                x1, y1, x2, y2 = map(int, boxes[0])

                # Crop the mask and the original image using the bounding box coordinates
                cropped_mask = mask[y1:y2, x1:x2]
                cropped_img = orig_img[y1:y2, x1:x2]

                # Convert cropped mask to binary image
                cropped_mask = (cropped_mask * 255).astype(np.uint8)

                # Create an RGBA image where the white part of the mask corresponds to the original image
                rgba_image = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2BGRA)
                rgba_image[:, :, 3] = cropped_mask

                # Save the RGBA image
                # processed_image_name = f"{os.path.splitext(os.path.basename(resized_image_paths[idx]))[0]}.png"
                # processed_image_path = f"{cfg.PROCESSED_IMAGES_PATH}/{island_name}/{lizard_name}/{processed_image_name}"
                processed_image_path = processed_image_paths[idx]
                print(f"Saving processed image to {processed_image_path}")
                cv2.imwrite(processed_image_path, rgba_image)

            except Exception as e:
                print(
                    f"Error processing lizard {lizard_path}: {e}. Continuing to the next image..."
                )
                images_with_errors.append(image_path)

        # Remove the temporary resized images
        for temp_image_path in resized_image_paths:
            os.remove(temp_image_path)

        # break  # Remove or modify this break to process all lizards
    # break  # Remove or modify this break to process all islands

Processing island Moltona



0: 640x640 1 body pattern, 3.9ms
Speed: 0.9ms preprocess, 3.9ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 body pattern, 1: 640x640 1 body pattern, 2: 640x640 1 body pattern, 3: 640x640 1 body pattern, 4: 640x640 1 body pattern, 8.5ms
Speed: 0.7ms preprocess, 1.7ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 body patterns, 4.6ms
Speed: 1.0ms preprocess, 4.6ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 body pattern, 1: 640x640 1 body pattern, 7.2ms
Speed: 0.9ms preprocess, 3.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


No images found for lizard M749



0: 640x640 2 body patterns, 1: 640x640 1 body pattern, 7.3ms
Speed: 1.0ms preprocess, 3.6ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 body pattern, 6.9ms
Speed: 1.2ms preprocess, 6.9ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 body pattern, 4.3ms
Speed: 0.9ms preprocess, 4.3ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 body pattern, 4.3ms
Speed: 0.9ms preprocess, 4.3ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 body pattern, 1: 640x640 1 body pattern, 2: 640x640 1 body pattern, 5.2ms
Speed: 0.8ms preprocess, 1.7ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 body pattern, 1: 640x640 1 body pattern, 2: 640x640 1 body pattern, 4.8ms
Speed: 0.8ms preprocess, 1.6ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 body pattern, 1: 640x640 1 body pattern, 5.8ms
Speed: 0.7ms preprocess

No images found for lizard M782


...


0: 640x640 1 body pattern, 4.3ms
Speed: 0.8ms preprocess, 4.3ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 body pattern, 1: 640x640 1 body pattern, 2: 640x640 1 body pattern, 3: 640x640 1 body pattern, 5.9ms
Speed: 0.6ms preprocess, 1.5ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 body pattern, 1: 640x640 1 body pattern, 4.6ms
Speed: 0.8ms preprocess, 2.3ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 body pattern, 4.6ms
Speed: 0.8ms preprocess, 4.6ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 body pattern, 1: 640x640 1 body pattern, 2: 640x640 1 body pattern, 3: 640x640 1 body pattern, 4: 640x640 1 body pattern, 7.0ms
Speed: 0.7ms preprocess, 1.4ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 body pattern, 4.7ms
Speed: 0.9ms preprocess, 4.7ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 640)

0: 

Error processing lizard N1468: 'NoneType' object has no attribute 'data'



0: 640x640 1 body pattern, 1: 640x640 1 body pattern, 2: 640x640 1 body pattern, 3: 640x640 2 body patterns, 4: 640x640 1 body pattern, 5: 640x640 1 body pattern, 6: 640x640 1 body pattern, 9.8ms
Speed: 0.6ms preprocess, 1.4ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 body pattern, 4.6ms
Speed: 0.8ms preprocess, 4.6ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 body pattern, 1: 640x640 1 body pattern, 2: 640x640 1 body pattern, 3: 640x640 1 body pattern, 4: 640x640 1 body pattern, 5: 640x640 1 body pattern, 6: 640x640 1 body pattern, 9.4ms
Speed: 0.6ms preprocess, 1.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 body pattern, 1: 640x640 1 body pattern, 2: 640x640 1 body pattern, 3: 640x640 1 body pattern, 4: 640x640 1 body pattern, 5: 640x640 1 body pattern, 10.4ms
Speed: 0.6ms preprocess, 1.7ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 body 

Error processing lizard N503: 'NoneType' object has no attribute 'data'



0: 640x640 1 body pattern, 1: 640x640 1 body pattern, 2: 640x640 1 body pattern, 3: 640x640 1 body pattern, 4: 640x640 1 body pattern, 7.0ms
Speed: 0.6ms preprocess, 1.4ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 body pattern, 4.6ms
Speed: 1.0ms preprocess, 4.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 body pattern, 4.2ms
Speed: 0.8ms preprocess, 4.2ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 body pattern, 1: 640x640 3 body patterns, 2: 640x640 1 body pattern, 3: 640x640 1 body pattern, 4: 640x640 1 body pattern, 5: 640x640 1 body pattern, 6: 640x640 1 body pattern, 7: 640x640 1 body pattern, 11.9ms
Speed: 0.6ms preprocess, 1.5ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 body pattern, 1: 640x640 1 body pattern, 2: 640x640 1 body pattern, 4.7ms
Speed: 0.6ms preprocess, 1.6ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 64

In [4]:
# Visualize the results
def visualize_results(result):
    # Plot the result image
    im_bgr = result.plot()  # BGR-order numpy array
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # Convert to RGB-order PIL image

    # Display the result using PIL's show method
    im_rgb.show()

In [29]:
images_with_errors
folder_with_errors = "images_with_errors"
os.makedirs(folder_with_errors, exist_ok=True)

# Copy images with errors to a folder
for image_path in images_with_errors:
    image_name = os.path.basename(image_path)
    print(f"Copying {image_name} to {folder_with_errors}")
    os.system(f"cp {image_path} {folder_with_errors}/{image_name}")

In [19]:
len(os.listdir(folder_with_errors))

73